<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fbigdatamagazine.es%2Fwp-content%2Fuploads%2F2023%2F02%2FFOTO-OK-BDM-DIG-DATA-MAGAZINE.jpg&f=1&nofb=1&ipt=4061921fa0da07483f83edb036d31f25545b2cae889c7eeefebd576f6e0fe5f4" style="width:300px; float: right; margin: 0 40px 40px 40px;"></img>


Al final de este paso, comprenderás los conceptos de **subajuste** (*underfitting*) y **sobreajuste** (*overfitting*), y serás capaz de aplicar estas ideas para hacer que tus modelos sean más precisos.

# Experimentar con Diferentes Modelos

Ahora que tienes una forma confiable de medir la precisión del modelo, puedes experimentar con modelos alternativos y ver cuál ofrece las mejores predicciones. Pero, ¿qué alternativas tienes?

Puedes consultar en la [documentación de scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) que el modelo de árbol de decisión tiene muchas opciones (más de las que vas a querer o necesitar durante un buen tiempo). Las opciones más importantes determinan la **profundidad del árbol**. Recuerda que la profundidad del árbol mide cuántas divisiones realiza antes de llegar a una predicción. Este es un árbol relativamente poco profundo:

![Árbol de profundidad 2](https://storage.googleapis.com/kaggle-media/learn/images/R3ywQsR.png)

En la práctica, no es raro que un árbol tenga 10 divisiones entre el nivel superior (todas las casas) y una hoja. A medida que el árbol se hace más profundo, el conjunto de datos se divide en hojas con menos viviendas. Si un árbol tiene solo 1 división, separa los datos en 2 grupos. Si cada grupo se divide de nuevo, obtendríamos 4 grupos de casas. Si volvemos a dividir cada uno de esos, se crean 8 grupos. Si continuamos duplicando el número de grupos agregando más divisiones en cada nivel, tendremos \\(2^{10}\\) grupos de viviendas al llegar al nivel 10. Eso son 1024 hojas.

Cuando dividimos las viviendas en muchas hojas, también hay menos casas por hoja. Las hojas con muy pocas casas generarán predicciones muy cercanas a los valores reales de esas viviendas, pero podrían ser predicciones muy poco confiables para nuevos datos (porque se basan en muy pocas observaciones).

Este fenómeno se llama **sobreajuste** (*overfitting*), donde un modelo se ajusta casi perfectamente a los datos de entrenamiento, pero tiene un rendimiento deficiente en los datos de validación y en datos nuevos. Por otro lado, si hacemos nuestro árbol muy poco profundo, no divide las viviendas en grupos realmente distintos.

En un caso extremo, si un árbol divide las viviendas en solo 2 o 4 grupos, cada grupo aún contendría una gran variedad de viviendas. Las predicciones resultantes podrían estar muy alejadas del valor real para la mayoría de las casas, incluso en los datos de entrenamiento (y también serían malas en los datos de validación por la misma razón). Cuando un modelo no logra capturar distinciones y patrones importantes en los datos, y por tanto tiene un mal desempeño incluso en entrenamiento, a eso se le llama **subajuste** (*underfitting*).

Dado que nos importa la precisión sobre datos nuevos —que estimamos usando los datos de validación— queremos encontrar el punto ideal entre subajuste y sobreajuste. Visualmente, queremos llegar al punto más bajo de la curva (roja) de validación en la siguiente figura:

![subajuste_sobreajuste](https://storage.googleapis.com/kaggle-media/learn/images/AXSEOfI.png)

# Ejemplo

Existen varias alternativas para controlar la profundidad del árbol, y muchas permiten que algunas ramas del árbol tengan mayor profundidad que otras. Pero el argumento *max_leaf_nodes* proporciona una forma muy razonable de controlar el sobreajuste frente al subajuste. Cuantas más hojas permitamos que el modelo cree, más nos desplazamos desde la zona de subajuste hacia la zona de sobreajuste en la gráfica anterior.

Podemos utilizar una función auxiliar para comparar los valores de MAE obtenidos con diferentes configuraciones del parámetro *max_leaf_nodes*:



In [3]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

Los datos se han cargado en **train_X**, **val_X**, **train_y** y **val_y** utilizando el código que ya has visto (y que ya has escrito).

In [1]:
import pandas as pd

# Cargar los datos
melbourne_file_path = '/content//melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path)

# Filtrar las filas con valores faltantes
filtered_melbourne_data = melbourne_data.dropna(axis=0)

# Seleccionar la variable objetivo (target) y las características (features)
y = filtered_melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea',
                      'YearBuilt', 'Lattitude', 'Longtitude']
X = filtered_melbourne_data[melbourne_features]

from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y validación, tanto para características como para la variable objetivo
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

Podemos utilizar un ciclo `for` para comparar la precisión de los modelos construidos con diferentes valores para *max_leaf_nodes*.

In [4]:
# Comparar el MAE con diferentes valores de max_leaf_nodes
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Máximo número de hojas: %d  \t\t Error Absoluto Medio:  %d" % (max_leaf_nodes, my_mae))

Máximo número de hojas: 5  		 Error Absoluto Medio:  347380
Máximo número de hojas: 50  		 Error Absoluto Medio:  258171
Máximo número de hojas: 500  		 Error Absoluto Medio:  243495
Máximo número de hojas: 5000  		 Error Absoluto Medio:  255575


De las opciones listadas, 500 es el número óptimo de hojas.

---

# Conclusión

Esto es lo que debes tener en cuenta: los modelos pueden verse afectados por:

- **Sobreajuste (overfitting):** cuando capturan patrones espurios que no se repetirán en el futuro, lo que lleva a predicciones menos precisas.
- **Subajuste (underfitting):** cuando no logran capturar patrones relevantes, también resultando en predicciones poco precisas.

Utilizamos datos de **validación**, que no se usan en el entrenamiento del modelo, para medir la precisión de un modelo candidato. Esto nos permite probar muchos modelos y conservar el que tenga el mejor desempeño.
